In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map.add_basemap("Esri.WorldImagery")
Map.add_basemap("OpenTopoMap")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")

In [4]:
dem_vis = {
'min': 0,
'max': 4000,
'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

landsat_vis = {
    'min': 20,
    'max': 200,
    'bands': ['B4', 'B3', 'B2']
}

In [5]:
Map.addLayer(dem, dem_vis, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(landsat7, landsat_vis, 'Landsat 7')
Map.addLayer(states, {}, "US States")

In [ ]:
import ee
import geemap
import numpy as np
import os

# Initialize Earth Engine
ee.Initialize()

# Define center point
center_lon = -87.63
center_lat = 41.87

# Calculate degree offsets for a true square
distance_km = 0.2  # desired width/height in kilometers
km_per_degree_lat = 111.32  # One degree of latitude in km
km_per_degree_lon = 111.32 * \
    np.cos(np.radians(center_lat))  # Adjust for latitude

lat_offset = distance_km / km_per_degree_lat
lon_offset = distance_km / km_per_degree_lon

# Create bounding box
bbox = ee.Geometry.Rectangle([
    center_lon - lon_offset,  # min_lon
    center_lat - lat_offset,  # min_lat
    center_lon + lon_offset,  # max_lon
    center_lat + lat_offset   # max_lat
])

# Get Sentinel-2 imagery
s2 = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(bbox)\
    .filterDate('2020-06-01', '2020-06-22')\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)\
    .select(['B2', 'B3', 'B4', 'B8'])\
    .mosaic()\
    .clip(bbox)

# Create & display map
Map = geemap.Map(center=[center_lat, center_lon], zoom=13)

# Add Sentinel image
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(s2, vis_params, 'Sentinel-2 Image')

# Add the bounding box
Map.addLayer(bbox, {'color': 'red'}, 'Bounding Box')
Map.centerObject(bbox)
    
display(Map)

# Create output directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Download the image
filename = 'data/chicago_loop_sentinel2.tif'
geemap.ee_export_image(
    s2,
    filename=filename,
    scale=10,  # 10m resolution
    region=bbox
)

print(f"Image downloaded to {filename}")
print(f"Box dimensions: {distance_km:.2f}km x {distance_km:.2f}km")
print(f"At 10m Sentinel-2 resolution, this should be approximately {
      int(distance_km*100)}x{int(distance_km*100)} pixels")

/home/dan/ordinaryevidence/fwi/fwi-innovation-challenge/env/lib/python3.12/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[41.87, -87.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

Generating URL ...
Please wait ...
Data downloaded to /home/dan/ordinaryevidence/fwi/fwi-innovation-challenge/data/chicago_loop_sentinel2.tif
Image downloaded to data/chicago_loop_sentinel2.tif
Box dimensions: 0.20km x 0.20km
At 10m Sentinel-2 resolution, this should be approximately 20x20 pixels
